In [1]:
import numpy as np
import pandas as pd
import polars as pl
from great_tables import GT, md

In [2]:
indicadores_fofisp = pd.read_csv('indicadores_fofisp.csv')
indicadores_fofisp['Categoría'] = indicadores_fofisp['Categoría'].fillna('')
indicadores_fofisp['Ponderación_categoría'] = indicadores_fofisp['Ponderación_categoría'].fillna(0)

In [3]:
(
    GT(indicadores_fofisp)
    .tab_stub()
    .tab_header(
        title='Indicadores de Distribución',
        subtitle='Fondo para el Fortalecimiento de las Instituciones de Seguridad Pública')
    .fmt_currency(columns=['Monto_asignado'])
    .fmt_percent(columns=['Ponderación_categoría','Ponderación_indicador'], decimals=1).sub_zero(zero_text=md(''))
    .cols_width(cases={
            "Categoría": "28%",
            "Ponderación_categoría": "15%",
            "Indicador": "32%",
            "Ponderación_indicador": "15%",
            "Monto_asignado": "10%"
            })
    .tab_source_note(
        source_note=md("Fuente: *Secretariado Ejecutivo del Sistema Nacional de Seguridad Pública*")
    )
)

GT(_tbl_data=                     Categoría  Ponderación_categoría  \
0                    Población                   0.75   
1  Capacidades Institucionales                   0.25   
2                                                0.00   
3                                                0.00   

                             Indicador  Ponderación_indicador  Monto_asignado  
0   Población en la Entidad Federativa                 0.7500    8.665824e+08  
1  Disminución de Incidencia Delictiva                 0.0833    9.628694e+07  
2      Incremento del Estado de Fuerza                 0.0833    9.628694e+07  
3  Instituciones de Profesionalización                 0.0833    9.628694e+07  , _body=<great_tables._gt_data.Body object at 0x7331e511cbf0>, _boxhead=Boxhead([ColInfo(var='Categoría', type=<ColInfoTypeEnum.default: 1>, column_label='Categoría', column_align='left', column_width='28%'), ColInfo(var='Ponderación_categoría', type=<ColInfoTypeEnum.default: 1>, column_label='Ponderación_categoría', column_align='right', column_width='15%'), ColInfo(var='Indicador', type=<ColInfoTypeEnum.default: 1>, column_label='Indicador', column_align='left', column_width='32%'), ColInfo(var='Ponderación_indicador', type=<ColInfoTypeEnum.default: 1>, column_label='Ponderación_indicador', column_align='right', column_width='15%'), ColInfo(var='Monto_asignado', type=<ColInfoTypeEnum.default: 1>, column_label='Monto_asignado', column_align='right', column_width='10%')]), _stub=<great_tables._gt_data.Stub object at 0x7331d1d147d0>, _spanners=Spanners([]), _heading=Heading(title='Indicadores de Distribución', subtitle='Fondo para el Fortalecimiento de las Instituciones de Seguridad Pública', preheader=None), _stubhead=None, _source_notes=[Md(text='Fuente: *Secretariado Ejecutivo del Sistema Nacional de Seguridad Pública*')], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7331e41ad0a0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x733258ce1760>, <great_tables._gt_data.FormatInfo object at 0x7331d1d14680>], _substitutions=[<great_tables._gt_data.FormatInfo object at 0x7331d1d149b0>], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none

In [4]:
w_pob = 0.75
w_var_edo_fza = 0.1
w_var_incidencia_del = 0.1
w_academias = 0.05

In [5]:
weights = {
    'Población': w_pob,
    'Var_edo_fza': w_var_edo_fza,
    'Var_incidencia_del': w_var_incidencia_del,
    'Academias': w_academias,
}

In [6]:
fofisp_datos_entrada = pd.read_csv('fofisp_datos_entrada.csv')

In [7]:
# --- Funciones de Cálculo del Índice ---
def min_max_normalize(series, direction='positive'):
    """
    Normaliza una serie de datos entre 0 y 1 usando el método Min-Max.
    Si la dirección es 'negativa', se invierte (Alto = Malo se convierte en Alto = Bueno).
    """
    min_val = series.min()
    max_val = series.max()

    if max_val == min_val:
        return pd.Series(0.5, index=series.index) # Retorna 0.5 si todos los valores son iguales

    if direction == 'positive':
        # (X - Min) / (Max - Min) -> Un valor más alto resulta en una puntuación más alta
        return (series - min_val) / (max_val - min_val)
    elif direction == 'negative':
        # (Max - X) / (Max - Min) -> Un valor más bajo (mejor) resulta en una puntuación más alta
        return (max_val - series) / (max_val - min_val)
    else:
        raise ValueError("La dirección debe ser 'positiva' o 'negativa'")

def calculate_index(df, weights):
    """Calcula el Índice Compuesto Normalizado."""

    # 1. Normalización de Variables
    # variables positivas
    df['Pob_norm'] = min_max_normalize(df['Población'], direction='positive')
    df['Var_edo_fza_norm'] = min_max_normalize(df['Var_edo_fza'], direction='positive')
    df['Academias_norm'] = min_max_normalize(df['Academias'], direction='positive')

    # variables negativas (menos es mejor, por lo tanto, se invierte)
    df['Var_incidencia_del_norm'] = min_max_normalize(df['Var_incidencia_del'], direction='negative')

    # 2. Aplicación de Ponderadores
    df['Indice Normalizado'] = (
        df['Pob_norm'] * weights['Población'] +
        df['Var_edo_fza_norm'] * weights['Var_edo_fza'] +
        df['Var_incidencia_del_norm'] * weights['Var_incidencia_del'] +
        df['Academias_norm'] * weights['Academias']
    )

    # El índice final también se normaliza a un rango de 0 a 1 para asegurar comparabilidad
    df['Indice Final (0-1)'] = min_max_normalize(df['Indice Normalizado'], direction='positive')
    epsilon = 0.05
    df['Indice Final (Corrimiento)'] = (df['Indice Final (0-1)'] * (1 - epsilon)) + epsilon

    return df

# --- Cálculo y Visualización ---
# Calcular el índice
df_results = calculate_index(fofisp_datos_entrada, weights)


In [8]:
presupuesto = 1_155_443_263.97

In [9]:
df_results.head(2)

,Entidad_Federativa,Población,Var_incidencia_del,Var_edo_fza,Academias,Asignacion_2025,Pob_norm,Var_edo_fza_norm,Academias_norm,Var_incidencia_del_norm,Indice Normalizado,Indice Final (0-1),Indice Final (Corrimiento)
0,Aguascalientes,1567559,-0.052055,0,70,17346435,0.046685,0.5,0.179487,0.590065,0.152995,0.049619,0.097138
1,Baja California,4189285,-0.206980,0,170,31213337,0.200444,0.5,0.435897,1.000000,0.322128,0.260871,0.297827


In [10]:
total_indice = df_results['Indice Final (Corrimiento)'].sum()

df_results['reparto'] = df_results['Indice Final (Corrimiento)'] / total_indice

In [11]:
total_indice

7.5247991731237

In [18]:
# reckon end allocated amount
# create share weights
df_results['Reparto'] = df_results['Indice Final (Corrimiento)'] / total_indice
# Var%funds
df_results['importe_asignado'] = df_results['Reparto'] * presupuesto

In [19]:
df_results['importe_asignado'].sum()

1155443263.97

In [23]:
# create diff amount and percentage
df_results['Var%'] = df_results['importe_asignado'] / df_results['Asignacion_2025'] -1

In [42]:
(
        df_results[['Entidad_Federativa','Indice Normalizado','Indice Final (0-1)','Indice Final (Corrimiento)',
                'importe_asignado','Asignacion_2025','Var%']]
                .style
                .format({
                        'importe_asignado': '${:,.2f}',
                        'Asignacion_2025': '${:,.2f}',
                        'Var%': '{:.2%}',
                        'Indice Normalizado':'{:.4f}',
                        'Indice Final (0-1)':'{:.4f}',
                        'Indice Final (Corrimiento':'{:.4f}',
                        })
)

,Entidad_Federativa,Indice Normalizado,Indice Final (0-1),Indice Final (Corrimiento),importe_asignado,Asignacion_2025,Var%
0,Aguascalientes,0.1530,0.0496,0.097138,"$14,915,619.17","$17,346,435.00",-14.01%
1,Baja California,0.3221,0.2609,0.297827,"$45,731,794.36","$31,213,337.00",46.51%
2,Baja California Sur,0.1145,0.0015,0.051436,"$7,897,994.15","$15,386,517.00",-48.67%
3,Campeche,0.1678,0.0681,0.114653,"$17,605,087.55","$15,590,662.00",12.92%
4,Coahuila,0.2365,0.1539,0.196246,"$30,133,799.13","$29,395,534.00",2.51%
5,Colima,0.1133,0.0000,0.050000,"$7,677,568.78","$28,411,222.00",-72.98%
6,Chiapas,0.3424,0.2862,0.321849,"$49,420,340.08","$44,183,005.00",11.85%
7,Chihuahua,0.2682,0.1935,0.233840,"$35,906,498.89","$33,623,684.00",6.79%
8,Ciudad de México,0.4966,0.4788,0.504849,"$77,520,317.14","$67,962,566.00",14.06%
9,Durango,0.1620,0.0608,0.107781,"$16,549,875.35","$30,804,419.00",-46.27%
